In [81]:
import dask
dask.config.set({"dataframe.query-planning": False})

from sklearn.model_selection import train_test_split, permutation_test_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
import anndata as ad
import numpy as np
import pandas as pd
import sys
import argparse
from pathlib import Path

sys.path.append("../../../workflow/scripts/")
import general_utils
import readwrite

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

# Access the arguments
sample_dir = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/baysor/breast/breast/0OE1/0OE1/')
sample_counts = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/std_seurat_analysis/baysor/breast/breast/0OE1/0OE1/lognorm/normalised_counts/data.parquet')
sample_idx = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/std_seurat_analysis/baysor/breast/breast/0OE1/0OE1/lognorm/normalised_counts/cells.parquet')
cell_type_labels = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/cell_type_annotation/baysor/breast/breast/0OE1/0OE1/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2/single_cell/labels.parquet')
proseg_results_dir = sample_dir / 'raw_results'
xeniumranger_dir = sample_dir / 'normalised_results/outs'

max_iter = 1000
n_neighbors = 10
n_permutations = 30
n_repeats = 5
cti = "macrophage"
ctj = "malignant cell of breast"


# read raw data to get spatial coordinates
adata = readwrite.read_xenium_sample(
    xeniumranger_dir,
    cells_as_circles=False,
    cells_boundaries=False,
    cells_boundaries_layers=False,
    nucleus_boundaries=False,
    cells_labels=False,
    nucleus_labels=False,
    transcripts=False,
    morphology_mip=False,
    morphology_focus=False,
    aligned_images=False,
    anndata=True,
)

# read normalised data
X_normalised = pd.read_parquet(sample_counts)
X_normalised.index = pd.read_parquet(sample_idx).iloc[:, 0]
adata = adata[X_normalised.index]
adata.X = X_normalised

# read labels
label_key = "label_key"
adata.obs[label_key] = pd.read_parquet(cell_type_labels).set_index("cell_id").iloc[:, 0]

# define target (cell type j presence in kNN)
knnlabels = general_utils.get_knn_labels(adata,n_neighbors=n_neighbors,label_key=label_key,knn_key='spatial')
adata.obs[f"has_{ctj}_neighbor"] = knnlabels[ctj]>0

# Filter for cti
if cti is None:
    adata_cti = adata
else:
    adata_cti = adata[adata.obs[label_key] == cti]


# Split data
X = adata_cti.X
y = adata_cti.obs[f"has_{ctj}_neighbor"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

# Baseline model (predict most frequent class)
# baseline_accuracy = max(np.bincount(y_test) / len(y_test))

# Init logistic regression model
model = LogisticRegression(max_iter=max_iter)

# Empirical p-value calculation using permutation test
score, permutation_scores, p_value = permutation_test_score(
    model, X_train, y_train, scoring="f1", n_permutations=n_permutations, n_jobs=-1
)

permutation_summary = pd.DataFrame(
    [[score, permutation_scores.mean(), permutation_scores.std(), p_value]],
    columns=['f1_score', 'permutation_mean_f1_score', 'permutation_std_f1_score', 'p_value'])
permutation_summary['effect_size'] =   (permutation_summary['f1_score'] - permutation_summary['permutation_mean_f1_score'] ) / permutation_summary['permutation_std_f1_score']

# Feature importances 
# sscaler = sklearn.preprocessing.StandardScaler(with_mean=False).fit(X)
# np.std(X, axis=0) * np.vstack([m.coef_[0] for m in cv_results['estimator']])
# cv_results = cross_validate(model,X,y,return_estimator=True, scoring="f1", n_jobs=-1)
model.fit(X_train, y_train)
importances = permutation_importance(model, pd.DataFrame.sparse.from_spmatrix(X_test), y_test, scoring='f1', n_repeats=n_repeats,n_jobs=-1)
importances.pop('importances')
importances = pd.DataFrame(importances,index=adata_cti.var_names).sort_values('importances_mean',ascending=False)

# Save
# permutation_summary.to_parquet(sample_dir / 'permutation_summary.parquet')
# importances.to_frame().to_parquet(sample_dir / 'importances.parquet')

INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/baysor/breast/br
         east/0OE1/0OE1/normalised_results/outs/cell_feature_matrix.h5                                             


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/tmp/ipykernel_82843/1494196430.py:56: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[label_key] = pd.read_parquet(cell_type_labels).set_index("cell_id").iloc[:, 0]


Baseline accuracy: 0.5499412455934195
Original Score: 0.626
Empirical p-value: 0.03225806451612903
Permutation Scores: 0.393 +/- 0.012


In [ ]:
import scipy
# Define the parameters
M = 1000  # Total number of features
n = 50    # Number of features in the specific group
N = 100   # Number of top features
k = 15    # Number of top features that belong to the specific group

# Hypergeometric test
p_value_hyper = scipy.stats.hypergeom.sf(k-1, M, n, N)
print(f"Hypergeometric test p-value: {p_value_hyper}")

Hypergeometric test p-value: 4.200127106596174e-05


In [ ]:
import scipy
# Define the parameters
M = 1000  # Total number of features
n = 50    # Number of features in the specific group
N = 100   # Number of top features
k = 15    # Number of top features that belong to the specific group

# Hypergeometric test
p_value_hyper = scipy.stats.hypergeom.sf(k-1, M, n, k)
print(f"Hypergeometric test p-value: {p_value_hyper}")

Hypergeometric test p-value: 4.200127106596174e-05


In [134]:
import gseapy

enrich = gseapy.enrich(gene_list=list(importances.index),gene_sets={'marker_genes':importances.index[-10:]})
enrich.results

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score,Genes
0,gs_ind_0,marker_genes,10/10,1.0,1.0,21.0,-0.0,CD14;TCF4;LPXN;C1QC;TRAF4;DSC2;HAVCR2;SCD;THAP...


In [135]:
enrich = gseapy.gsea(gene_list=list(importances.index),gene_sets={'marker_genes':importances.index[-10:]})
enrich.results

TypeError: gsea() missing 2 required positional arguments: 'data' and 'cls'

In [ ]:
from scipy.stats import hypergeom

# Define the parameters
N = 1000  # Total number of features
K = 50    # Number of features in the specific group
n = 100   # Number of top features
k = 15    # Number of top features that belong to the specific group

# Hypergeometric test
p_value_hyper = hypergeom.sf(k-1, N, K, n)
print(f"Hypergeometric test p-value: {p_value_hyper}")

In [5]:
import dask
dask.config.set({"dataframe.query-planning": False})

import spatialdata_io
import numpy as np
import pandas as pd
import scvi
import sys
import argparse
from pathlib import Path

sys.path.append("../../../workflow/scripts/")
import preprocessing
import readwrite

# Access the arguments
sample_dir = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/proseg/breast/breast/0OE1/0OE1')
proseg_results_dir = sample_dir / 'raw_results'
xeniumranger_dir = sample_dir / 'normalised_results/outs'
cell_type_labels = None
min_counts = 5
min_features = 5
max_counts = np.inf
max_features = np.inf
min_cells = 5

dir_resolvi_model='model'
num_samples=30
batch_size=1000


# read counts
adata = readwrite.read_xenium_sample(
    xeniumranger_dir,
    cells_as_circles=False,
    cells_boundaries=False,
    cells_boundaries_layers=False,
    nucleus_boundaries=False,
    cells_labels=False,
    nucleus_labels=False,
    transcripts=False,
    morphology_mip=False,
    morphology_focus=False,
    aligned_images=False,
    anndata=True,
)

# need to round proseg expected counts for resolVI to run
# no need for if statement, doesn't change anything to other segmentation methods
adata.X.data = adata.X.data.astype(np.float32).round()
adata.obs_names = adata.obs_names.astype(str)

if cell_type_labels is not None:
    labels_key = "labels_key"
    semisupervised = True
    adata.obs[labels_key] = pd.read_parquet(cell_type_labels).iloc[:, 0]
else:
    labels_key = None
    semisupervised = False

# preprocess (QC filters only)
# resolvi requires at least 5 counts in each cell
preprocessing.preprocess(
    adata,
    normalize=False,
    log1p=False,
    scale="none",
    pca=False,
    umap=False,
    save_raw=False,
    min_counts=min_counts,
    min_genes=min_features,
    max_counts=max_counts,
    max_genes=max_features,
    min_cells=min_cells,
    backend="cpu",
)
resolvi = scvi.external.RESOLVI.load(dir_resolvi_model, adata=adata)

samples_corr = resolvi.sample_posterior(
    model=resolvi.module.model_corrected,
    return_sites=["px_rate"],
    summary_fun={"post_donor_q50": np.median},
    num_samples=num_samples,
    batch_size=batch_size,
    summary_frequency=100,
)
samples_corr = pd.DataFrame(samples_corr).T

samples = resolvi.sample_posterior(
    model=resolvi.module.model_residuals,
    return_sites=["mixture_proportions"],
    summary_fun={"post_donor_means": np.mean},
    num_samples=num_samples,
    batch_size=batch_size,
    summary_frequency=100,
)
samples_proportions = pd.DataFrame(samples).T

### save
samples_corr = pd.DataFrame(
    samples_corr.loc["post_donor_q50", "px_rate"],
    index=adata.obs_names,
    columns=adata.var_names,
)
samples_proportions = pd.DataFrame(
    samples_proportions.loc["post_donor_means", "mixture_proportions"],
    index=adata.obs_names,
    columns=["true_proportion", "diffusion_proportion", "background_proportion"],
)


INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/proseg/breast/br
         east/0OE1/0OE1/normalised_results/outs/cell_feature_matrix.h5                                             
Removed 1476  cells...
Removed 0  genes...
INFO     File model/model.pt already downloaded                                                                    


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/scanpy/neighbors/__init__.py:700: FutureWarning: `transformer='rapids'` is deprecated. Use `rapids_singlecell.tl.neighbors` instead.
  warn(msg, FutureWarning)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib

Epoch 1/50:   2%|▏         | 1/50 [00:00<00:03, 14.64it/s, v_num=1]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 1/50:   2%|▏         | 1/50 [00:00<00:03, 13.99it/s, v_num=1]

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...



Sampling local variables, batch: 100%|██████████| 68/68 [01:00<00:00,  1.13it/s]


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...


Sampling local variables, batch:  75%|███████▌  | 51/68 [00:27<00:09,  1.83it/s]


KeyboardInterrupt: 

In [ ]:
adata.layers["generated_expression"] = scp.csr_matrix(samples_corr.loc[sample_quantile, 'obs'])

In [ ]:
from pomegranate.distributions import Poisson
from pomegranate.gmm import GeneralMixtureModel
from matplotlib.gridspec import GridSpec
from matplotlib import pyplot as plt
import seaborn as sns
import torch
DPI = 100

def double_positive_pmm(adata, marker_list, marker_dict=None, layer_key='counts', output_dir="", file_save='', vmax=0.3):
    if True: # 'positive_pmm_counts' not in adata.obsm:
        adata.obsm['positive_pmm_counts'] = pd.DataFrame(
            0, index=adata.obs_names, columns=marker_list)
        for gene_x in marker_list:
            model = GeneralMixtureModel(
                [Poisson([0.001], inertia=0.99), Poisson([0.5], inertia=0.999)], verbose=False).cuda()
            model_fit = model.fit(
                torch.tensor(adata.obsm['counts'][gene_x].values[..., np.newaxis]).cuda())
            adata.obsm['positive_pmm_counts'][gene_x] = model_fit.predict(torch.tensor(adata.obsm['counts'][gene_x].values[..., np.newaxis]).cuda()).cpu()
    
    adata.obsm[f'positive_pmm_{layer_key}'] = pd.DataFrame(
            0, index=adata.obs_names, columns=marker_list)
    for gene_x in marker_list:
        model = GeneralMixtureModel(
            [Poisson([0.001], inertia=0.99), Poisson([0.5], inertia=0.999)], verbose=False).cuda()
        model_fit = model.fit(
                torch.tensor(adata.obsm[layer_key][gene_x].values[..., np.newaxis]).cuda())
        adata.obsm[f'positive_pmm_{layer_key}'][gene_x] = model_fit.predict(torch.tensor(adata.obsm[layer_key][gene_x].values[..., np.newaxis]).cuda()).cpu()
        
    adata.uns[f'double_positive_{layer_key}'] = pd.DataFrame(
        0, index=marker_list, columns=marker_list)
        
    for ind_x, gene_x in enumerate(marker_list):  
        for ind_y, gene_y in enumerate(marker_list):
            if ind_x<ind_y:
                positives = adata.obsm['positive_pmm_counts'][[gene_x, gene_y]].sum(1)
                adata.uns[f'double_positive_{layer_key}'].loc[gene_x, gene_y] = np.sum(positives==2) / np.sum(positives>0) if np.sum(positives)>0 else None
            else:
                positives = adata.obsm[f'positive_pmm_{layer_key}'][[gene_x, gene_y]].sum(1)
                adata.uns[f'double_positive_{layer_key}'].loc[gene_x, gene_y] = np.sum(positives==2) / np.sum(positives>0) if np.sum(positives)>0 else None
                
    fig = plt.figure(figsize=(40, 40), dpi=DPI)
    gs = GridSpec(1, 1)
    ax = fig.add_subplot(gs[0])
    sns.set(font_scale=1.) # for label size
    sns.heatmap(adata.uns[f'double_positive_{layer_key}'], annot=True, annot_kws={"size": 6},
                square=True, linecolor='black', fmt=".1%", mask=np.identity(len(marker_list)),
                cmap='viridis', linewidths=1.2, vmax=vmax, xticklabels=True, yticklabels=True, ax=ax)
    
    if marker_dict:
        current_pos = 0
        for group, genes in marker_dict.items():
            gene_count = len(genes)
            if gene_count > 0:
                position = (current_pos, current_pos + gene_count - 1)
                current_pos += gene_count
                ax.text((position[0] + position[1]) / 2, - 1.3, group, ha='center', va='bottom', color='navy', fontsize=12)
                ax.plot([position[0] + 0.5, position[1] + 0.5], [- 0.5, - 0.5], lw=4, c='navy')

        ax.set_ylim([len(marker_list), -2])
        plt.subplots_adjust(top=0.9, bottom=0.5, left=0.07, right=0.93, hspace=0.2, wspace=0.2)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    plt.savefig(f'{output_dir}/double_positives_{layer_key}{file_save}.pdf')